## TFT
- need to create feature like date and day of week. 

In [1]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import plotly.express as px
import numpy as np


In [2]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
nwp_data

<xarray.Dataset> Size: 1TB
Dimensions:    (variable: 14, init_time: 5049, step: 49, latitude: 241,
                longitude: 301)
Coordinates:
  * init_time  (init_time) datetime64[ns] 40kB 2017-01-01 ... 2023-12-31T12:0...
  * latitude   (latitude) float64 2kB 60.0 59.95 59.9 59.85 ... 48.1 48.05 48.0
  * longitude  (longitude) float64 2kB -12.0 -11.95 -11.9 ... 2.9 2.95 3.0
  * step       (step) timedelta64[ns] 392B 00:00:00 01:00:00 ... 2 days 00:00:00
  * variable   (variable) <U5 280B 't2m' 'dswrf' 'dlwrf' ... 'v10' 'u100' 'v100'
Data variables:
    ECMWF_UK   (variable, init_time, step, latitude, longitude) float32 1TB ...

In [3]:
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')
pv_data

<xarray.Dataset> Size: 2GB
Dimensions:   (datetime: 387254)
Coordinates:
  * datetime  (datetime) datetime64[ns] 3MB 2018-01-01T06:05:00 ... 2021-10-2...
Data variables: (12/1311)
    10003     (datetime) float32 2MB ...
    10004     (datetime) float32 2MB ...
    10005     (datetime) float32 2MB ...
    10048     (datetime) float32 2MB ...
    10049     (datetime) float32 2MB ...
    10063     (datetime) float32 2MB ...
    ...        ...
    9870      (datetime) float32 2MB ...
    9871      (datetime) float32 2MB ...
    9902      (datetime) float32 2MB ...
    9903      (datetime) float32 2MB ...
    9960      (datetime) float32 2MB ...
    9989      (datetime) float32 2MB ...

- For 50 pv_sites, we are selecting 1500 dates. Means 75000
- Then we select 48 hours for each, but not all dates have 48 hours. 
- My dataset is 17046
- It means not all ss_ids, have 1500 datetime, with 48 hours gap. because i am selecting random after 48 hours gap.

In [7]:
## proof as data poitns are missing and not all ss_id ahve data for all hours in the dates. 


import pandas as pd

# Define the specific ss_id and the datetime range
ss_id = '6641'
start_datetime = pd.Timestamp('2021-10-27 19:00:00')
end_datetime = pd.Timestamp('2021-10-29 18:59:00')


hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
# Select the data for the specified range and ss_id
selected_data = hourly_pv_data.sel(datetime=slice(start_datetime, end_datetime))

# Check available datetimes
available_datetimes = selected_data['datetime'].values
generation_values = selected_data[ss_id].values

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'datetime': available_datetimes,
    'generation': generation_values
})

print(results_df)


             datetime  generation
0 2021-10-27 19:00:00         0.0
1 2021-10-27 20:00:00         0.0
2 2021-10-27 21:00:00         0.0
3 2021-10-27 22:00:00         0.0
4 2021-10-27 23:00:00         0.0


Conclusion is that not all points  have all hourly data between 2 days, so it is quite possible if i select a point, it might have only next 5 hours, and not all 48 hours. So dataset would not be optimal 25 ss_ids to 50 datetimes and 48 steps. Multiplication won't work, because there are too many missing data points. 

In [8]:
# add lat, long, tilt orientation
meta_data = pd.read_csv("data_files/metadata.csv")
meta_data.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


In [9]:
def get_48_hour_range(start_datetime):
    end_datetime = start_datetime + pd.Timedelta(hours=47, minutes=59)
    return start_datetime, end_datetime

def select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours):
    selected_datetimes = []
    available_datetimes = list(datetimes)

    for _ in range(num_selections):
        if not available_datetimes:
            break
        random_datetime = np.random.choice(available_datetimes)
        selected_datetimes.append(random_datetime)
        
        # Remove datetimes within the min_gap_hours range
        available_datetimes = [dt for dt in available_datetimes if dt > random_datetime + pd.Timedelta(hours=min_gap_hours)]

    return selected_datetimes

In [11]:

hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
datetimes = pd.to_datetime(hourly_pv_data['datetime'].values)

data_vars = list(pv_data.data_vars)
pv_sites_id = np.random.choice(data_vars, 50, replace=False)
print(pv_sites_id)

data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : [], 'hour':[]}

num_selections = 1500
min_gap_hours = 48

for ss_id in pv_sites_id:
    selected_datetimes = select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours)
    
    for start_datetime in selected_datetimes:
        start, end = get_48_hour_range(start_datetime)
        selected_data = hourly_pv_data.sel(datetime=slice(start, end))
        
        hour_counter = 1
        for dt, power in zip(selected_data['datetime'].values, selected_data[ss_id].values):
            data_dict['ss_id'].append(int(ss_id))
            data_dict['pv_datetime'].append(dt)
            data_dict['generation'].append(power)
            data_dict['hour'].append(hour_counter)
            hour_counter += 1
                                           



['8558' '7132' '6643' '26943' '10586' '10792' '9816' '10631' '10528'
 '3242' '7201' '3454' '7932' '7557' '15051' '26836' '6917' '26826' '26874'
 '4323' '8715' '14394' '7085' '4045' '7195' '10438' '7390' '7344' '26947'
 '8420' '7836' '3147' '9960' '3266' '27023' '8856' '5780' '14649' '10693'
 '6971' '26899' '26999' '7338' '15846' '6049' '3326' '7693' '16267'
 '10532' '27008']


In [12]:


pv_df = pd.DataFrame(data_dict)
print(pv_df.shape)
pv_df = pv_df.dropna(subset={'generation'})

pv_df
print(pv_df.shape)


(15024, 4)
(10149, 4)


In [13]:
pv_df.head()

,ss_id,pv_datetime,generation,hour
0,8558,2019-01-03 12:00:00,79.708443,1
1,8558,2019-01-03 13:00:00,55.071362,2
2,8558,2019-01-03 14:00:00,33.447720,3
3,8558,2019-01-03 15:00:00,6.544188,4
4,8558,2019-01-03 16:00:00,0.000000,5


In [14]:
# pv_df.to_csv("Try.csv")

In [15]:
pv_sites_id = [int(id) for id in pv_sites_id]  
# pv_sites_id

In [16]:
pv_site_dict = {'ss_id':[], "lat":[], "long": [], 'tilt':[], 'orientation':[]}

for id in pv_sites_id:
    row = meta_data[meta_data['ss_id'] == id]
    if not row.empty:
        pv_site_dict['ss_id'].append(id)
        pv_site_dict['lat'].append(row['latitude_rounded'].values[0])
        pv_site_dict['long'].append(row['longitude_rounded'].values[0])
        pv_site_dict['tilt'].append(row['tilt'].values[0])
        pv_site_dict['orientation'].append(row['orientation'].values[0])
    else:
        print('row empty')
        


print(pv_site_dict)
meta_site_df = pd.DataFrame.from_dict(pv_site_dict)
# meta_site_df

{'ss_id': [8558, 7132, 6643, 26943, 10586, 10792, 9816, 10631, 10528, 3242, 7201, 3454, 7932, 7557, 15051, 26836, 6917, 26826, 26874, 4323, 8715, 14394, 7085, 4045, 7195, 10438, 7390, 7344, 26947, 8420, 7836, 3147, 9960, 3266, 27023, 8856, 5780, 14649, 10693, 6971, 26899, 26999, 7338, 15846, 6049, 3326, 7693, 16267, 10532, 27008], 'lat': [56.0, 51.26, 54.51, 51.34, 56.97, 54.49, 52.86, 52.07, 54.54, 56.43, 51.12, 51.65, 51.29, 51.73, 57.52, 53.34, 53.1, 51.59, 51.32, 51.3, 52.12, 55.82, 54.81, 55.64, 51.13, 54.52, 55.94, 51.42, 51.52, 55.77, 55.85, 51.12, 55.52, 51.7, 51.59, 50.39, 51.34, 55.74, 54.85, 51.55, 51.3, 51.38, 55.06, 54.32, 55.63, 51.54, 57.34, 51.68, 57.29, 51.3], 'long': [-2.52, 0.52, -3.54, -1.73, -2.23, -1.13, -3.04, -3.0, -1.3, -3.4, -3.0, 0.62, -1.11, 0.9, -1.82, -2.8, -0.17, -1.82, -2.96, -2.45, -4.23, -3.96, -2.44, -3.89, -3.1, -1.36, -4.77, -0.19, -0.42, -4.09, -4.59, -0.02, -4.08, -2.33, -1.8, -4.07, 1.32, -4.03, -1.59, -1.84, -2.21, -0.31, -1.46, -2.76, -4.49, -2

In [17]:
combined_df = pd.merge(pv_df, meta_site_df, on='ss_id', how='inner')
combined_df

,ss_id,pv_datetime,generation,hour,lat,long,tilt,orientation
0,8558,2019-01-03 12:00:00,7.970844e+01,1,56.0,-2.52,45.0,165.0
1,8558,2019-01-03 13:00:00,5.507136e+01,2,56.0,-2.52,45.0,165.0
2,8558,2019-01-03 14:00:00,3.344772e+01,3,56.0,-2.52,45.0,165.0
3,8558,2019-01-03 15:00:00,6.544188e+00,4,56.0,-2.52,45.0,165.0
4,8558,2019-01-03 16:00:00,0.000000e+00,5,56.0,-2.52,45.0,165.0
...,...,...,...,...,...,...,...,...
10144,27008,2021-10-27 16:00:00,1.033115e+02,44,51.3,-0.23,30.0,180.0
10145,27008,2021-10-27 17:00:00,5.429112e-08,45,51.3,-0.23,30.0,180.0
10146,27008,2021-10-27 18:00:00,0.000000e+00,46,51.3,-0.23,30.0,180.0
10147,27008,2021-10-27 19:00:00,0.000000e+00,47,51.3,-0.23,30.0,180.0


In [18]:
# combined_df.to_csv("test.csv")

In [69]:
# test = pd.read_csv("test.csv")
# test.head()

In [19]:
combined_df['pv_datetime'] = pd.to_datetime(combined_df['pv_datetime'])

# Extract date and hour from pv_datetime
combined_df['pv_date'] = combined_df['pv_datetime'].dt.date
combined_df['pv_hour'] = combined_df['pv_datetime'].dt.hour
combined_df

,ss_id,pv_datetime,generation,hour,lat,long,tilt,orientation,pv_date,pv_hour
0,8558,2019-01-03 12:00:00,7.970844e+01,1,56.0,-2.52,45.0,165.0,2019-01-03,12
1,8558,2019-01-03 13:00:00,5.507136e+01,2,56.0,-2.52,45.0,165.0,2019-01-03,13
2,8558,2019-01-03 14:00:00,3.344772e+01,3,56.0,-2.52,45.0,165.0,2019-01-03,14
3,8558,2019-01-03 15:00:00,6.544188e+00,4,56.0,-2.52,45.0,165.0,2019-01-03,15
4,8558,2019-01-03 16:00:00,0.000000e+00,5,56.0,-2.52,45.0,165.0,2019-01-03,16
...,...,...,...,...,...,...,...,...,...,...
10144,27008,2021-10-27 16:00:00,1.033115e+02,44,51.3,-0.23,30.0,180.0,2021-10-27,16
10145,27008,2021-10-27 17:00:00,5.429112e-08,45,51.3,-0.23,30.0,180.0,2021-10-27,17
10146,27008,2021-10-27 18:00:00,0.000000e+00,46,51.3,-0.23,30.0,180.0,2021-10-27,18
10147,27008,2021-10-27 19:00:00,0.000000e+00,47,51.3,-0.23,30.0,180.0,2021-10-27,19


## Matching pv data with NWP
Need to match pv_datetime, with nwp, split pv_datetime to date and time seperatly, match pv_date with nwp init_time, and pv_hour with nwp steps. 

- 12 is midnight, so need to match with previous time.  using `backfill`

In [20]:
dataframes = []

One approach can be that whereever the hour is not increment of last hour, reset the batch and get the steps. Assign each step with the next row in combined data, till hour is increment of the last one. 

If hour is not increment of last one, then stop, and match the date again, get the steps, but go till only hour is increment of last one, can use while loop. and when hour is not increment of laast one, reset it and get the dat again and do it again. 

In [46]:
# # approach is to match exact date and time with init_time, previous one,a nd then get all the next 48 steps from nwp data. 
# # take from pv_datetime, date and hour to match with nwp_init time, now go to next 48 rows in combined_dataset for the next 48 steps of pv_datetime
# # now match these 48 steps with 49 steps available in nwp data fro each init_time. 
# # and get the valeus of aviables for the next 48 steps and append to combine dataset. 

# # now conider that not all points have 48 hours continous data. so blocks can not be of 48 hours. 

# dataframes = []

# # Iterate over the combined_df in steps of 48 to process each 48-hour block
# for start_idx in range(0, len(combined_df), 48):
#     block = combined_df.iloc[start_idx:start_idx + 48]
#     if len(block) < 48 or not is_continuous(block):
#         print("Not a continuous 48-hour block:", block.iloc[0]['pv_datetime'])
#         continue   # Skip blocks that are not full 48-hour periods

#     ss_id = block.iloc[0]['ss_id']
#     init_time = block.iloc[0]['pv_datetime']
#     lat = block.iloc[0]['lat']
#     lon = block.iloc[0]['long']
#     generation_values = block['generation'].values
#     pv_datetimes = block['pv_datetime'].values
#     tilt = block.iloc[0]['tilt']
#     orientation = block.iloc[0]['orientation']

#     # Match pv_datetime to the nearest previous init_time
#     init_time_sel = nwp_data.sel(init_time=init_time, method="backfill")
    
#     # Retrieve the 48 steps of forecast data starting from the matched init_time
#     data_sel = init_time_sel.sel(step=slice(pd.Timedelta(hours=0), pd.Timedelta(hours=47)))
#     data_sel = data_sel.sel(latitude=lat, method="nearest") \
#                        .sel(longitude=lon, method="nearest")
    
#     # Convert to DataFrame and pivot
#     data_df = data_sel.to_dataframe().reset_index()
#     pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

#     # Add the combined dataset columns
#     for i in range(48):
#         pivot_df.loc[i, 'ss_id'] = ss_id
#         pivot_df.loc[i, 'pv_datetime'] = pv_datetimes[i]
#         pivot_df.loc[i, 'generation'] = generation_values[i]

#     dataframes.append(pivot_df)
    



In [21]:
len(dataframes)

0

In [ ]:
# print(len(dataframes))
# # Combine all dataframes into one
# forecast_df = pd.concat(dataframes, ignore_index=True)

# # Display the final DataFrame
# print(forecast_df)

In [28]:
# approach is to match exact date of pvdate with init time date, adn then match exact time of pv hour with step hour. 

dataframes = []

for index, row in combined_df.iterrows():

    ss_id = row['ss_id']

    # Match pv_date to init_time
    init_time_sel = nwp_data.sel(init_time=row['pv_datetime'], method="backfill")
    
    # Match pv_hour to step
    step_sel = init_time_sel.sel(step=pd.Timedelta(hours=row['pv_hour']), method="nearest")
    
    data_sel = step_sel.sel(latitude=row['lat'], method="nearest") \
                       .sel(longitude=row['long'], method="nearest")
    

    data_df = data_sel.to_dataframe().reset_index()
    pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

    pivot_df['ss_id'] = ss_id
    pivot_df['pv_datetime'] = row['pv_datetime']
    pivot_df['generation'] = row['generation']
    pivot_df['pv_hour'] = row['pv_hour']
    pivot_df['tilt'] = row['tilt']
    pivot_df['orientation'] = row['orientation']
    pivot_df['lat'] = row['lat']
    pivot_df['long'] = row['long']
    pivot_df['hour'] = row['hour']
    

    dataframes.append(pivot_df)

In [23]:
# dataframes
print(len(dataframes))

10149


In [29]:
final_df = pd.concat(dataframes, ignore_index=True)
desired_order = ['ss_id', 'init_time', 'step', 'generation', 'pv_datetime', 'pv_hour', 'hour', 'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100']
final_df = final_df[desired_order]
print(final_df.shape)

(10149, 25)


In [30]:
final_df

variable,ss_id,init_time,step,generation,pv_datetime,pv_hour,hour,lat,long,tilt,...,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100
0,8558,2019-01-03 12:00:00,0 days 12:00:00,7.970844e+01,2019-01-03 12:00:00,12,1,56.0,-2.52,45.0,...,0.429790,0.000000,0.0,3066112.0,276.049805,0.522544,4.276141,7.151619,1.629375,1.932225
1,8558,2019-01-04 00:00:00,0 days 13:00:00,5.507136e+01,2019-01-03 13:00:00,13,2,56.0,-2.52,45.0,...,0.326599,0.000031,0.0,5102080.0,279.032471,0.726959,6.296407,8.906005,0.426015,0.372606
2,8558,2019-01-04 00:00:00,0 days 14:00:00,3.344772e+01,2019-01-03 14:00:00,14,3,56.0,-2.52,45.0,...,0.083099,0.000015,0.0,6565120.0,279.077148,0.398285,6.263617,9.459293,0.253472,0.000977
3,8558,2019-01-04 00:00:00,0 days 15:00:00,6.544188e+00,2019-01-03 15:00:00,15,4,56.0,-2.52,45.0,...,0.086731,0.000000,0.0,7739904.0,278.866943,0.092560,5.691016,8.951658,0.432409,0.246782
4,8558,2019-01-04 00:00:00,0 days 16:00:00,0.000000e+00,2019-01-03 16:00:00,16,5,56.0,-2.52,45.0,...,0.069794,0.000031,0.0,8128000.0,278.349365,0.069794,5.302433,8.770031,0.857510,0.785898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10144,27008,2021-10-28 00:00:00,0 days 16:00:00,1.033115e+02,2021-10-27 16:00:00,16,44,51.3,-0.23,30.0,...,0.000000,0.000000,0.0,21140992.0,286.916016,0.276733,-0.350718,-0.152166,6.057666,10.099644
10145,27008,2021-10-28 00:00:00,0 days 17:00:00,5.429112e-08,2021-10-27 17:00:00,17,45,51.3,-0.23,30.0,...,0.000000,0.000000,0.0,21553152.0,285.946533,0.910919,-0.520894,-0.319089,5.903310,10.377252
10146,27008,2021-10-28 00:00:00,0 days 18:00:00,0.000000e+00,2021-10-27 18:00:00,18,46,51.3,-0.23,30.0,...,0.000000,0.000000,0.0,21553152.0,285.897705,0.999298,-0.821718,-0.721787,6.102237,10.780716
10147,27008,2021-10-28 00:00:00,0 days 19:00:00,0.000000e+00,2021-10-27 19:00:00,19,47,51.3,-0.23,30.0,...,0.000275,0.000000,0.0,21553152.0,286.163086,0.999878,-0.918021,-1.002132,6.466412,11.268894


In [31]:
final_df.columns

Index(['ss_id', 'init_time', 'step', 'generation', 'pv_datetime', 'pv_hour',
       'hour', 'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf', 'duvrs',
       'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10',
       'v100'],
      dtype='object', name='variable')

In [32]:
final_df.to_csv("result_data/final_data_tft2.csv")

In [ ]:
## plots of generation and weather variables to see the trends
## add titl and orientation in dataset

## Wrong Approach
Instead of matching each datetime and hour with every datetime and steps. 
We need to select only one pv datetime to one nwp init_time, and then use next 48 hours from it.
Not exact matching with datetime. 

We need to get 48 next hours prediction. 

Try this as well, but focus more on model training. 
